<img src=http://fd.perso.eisti.fr/Logos/TORUS2.png>

In this section, let us present to you some Machine Learning algorithms, there are many, but 3 algorithms below can be considered as the most popular in Machine Learning :

- 1/ Regression - Linear Regression
- 2/ Classification - Random Forest
- 3/ Clustering - KMeans

This notebook will focus on the first one, we'll take a dataset and then build a linear regression model based on it. 

"Linear regression is the most basic type of regression and commonly used predictive analysis.  The overall idea of regression is to examine two things: (1) does a set of predictor variables do a good job in predicting an outcome variable?  Is the model using the predictors accounting for the variability in the changes in the dependent variable? (2) Which variables in particular are significant predictors of the dependent variable?  And in what way do they--indicated by the magnitude and sign of the beta estimates--impact the dependent variable?  These regression estimates are used to explain the relationship between one dependent variable and one or more independent variables. (3) What is the regression equation that shows how the set of predictor variables can be used to predict the outcome?  The simplest form of the equation with one dependent and one independent variable is defined by the formula y = c + b*x, where y = estimated dependent score, c = constant, b = regression coefficients, and x = independent variable."

(source : http://www.statisticssolutions.com/what-is-linear-regression/)

### Read dataset (csv format) from HDFS

Here we use the dataset from http://www.statsci.org/data/general/water.html 

The target variable will be monthly water usage (gallons) and the variables descriptives are : 
- Average monthly temperature (F)
- Amount of production (M pounds)
- Number of plant operating days in the month
- Number of persons on the monthly plant payroll

In [ ]:
val sqlContext = new SQLContext(sc)

val data = sqlContext.read.format("com.databricks.spark.csv")
              .option("header", "true").option("inferSchema", "true") 
              .load("hdfs://hupi-factory-02-01-01-01/user/hupi/dataset_torusVN/formation4_ML/water.csv")

<console>:69: warning: constructor SQLContext in class SQLContext is deprecated: Use SparkSession.builder instead
       val sqlContext = new SQLContext(sc)
                        ^
sqlContext: org.apache.spark.sql.SQLContext = org.apache.spark.sql.SQLContext@d717e65
data: org.apache.spark.sql.DataFrame = [Temperature: double, Production: int ... 3 more fields]


In [ ]:
data.show()

In [ ]:
data.printSchema()

### Some descriptions of data

#### Statistics summary 

In [ ]:
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.stat.{MultivariateStatisticalSummary, Statistics}

In [ ]:
// Convert df to RDD to be able to use the library MultiVariateStatisticalSummary.
val rdd = data.map(l => (l(0).asInstanceOf[Double], l(1).asInstanceOf[Integer].toDouble, l(2).asInstanceOf[Integer].toDouble,
                        l(3).asInstanceOf[Integer].toDouble, l(4).asInstanceOf[Integer].toDouble)).rdd

In [ ]:
rdd.take(2)

entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
// Convert rdd to the rdd of vectors
val observations = rdd.map(l => Vectors.dense(l._1, l._2, l._3, l._4, l._5))

In [ ]:
// Compute column summary statistics.
val summary: MultivariateStatisticalSummary = Statistics.colStats(observations)
println("Vectors of observations' mean : " + summary.mean)  
println("Vectors of observations' variance : " + summary.variance)  
println("Vectors of observations' number of column not null : " + summary.numNonzeros)  
println()

#### Correlations of variables 

In [ ]:
import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.stat.Statistics
import org.apache.spark.rdd.RDD

import org.apache.spark.mllib.linalg._
import org.apache.spark.mllib.stat.Statistics
import org.apache.spark.rdd.RDD


In [ ]:
// calculate the correlation matrix using Pearson's method. Use "spearman" for Spearman's method
// If a method is not specified, Pearson's method will be used by default.
val correlMatrix: Matrix = Statistics.corr(observations, "pearson")
println(correlMatrix.toString)

1.0                   -0.02410741870356305  0.43762975958335126   ... (5 total)
-0.02410741870356305  1.0                   0.10573054707596519   ...
0.43762975958335126   0.10573054707596519   1.0                   ...
-0.08205777488270032  0.9184797375869633    0.03188119325449726   ...
0.28575755805713965   0.6307494802500775    -0.08882582642644302  ...
correlMatrix: org.apache.spark.mllib.linalg.Matrix =
1.0                   -0.02410741870356305  0.43762975958335126   ... (5 total)
-0.02410741870356305  1.0                   0.10573054707596519   ...
0.43762975958335126   0.10573054707596519   1.0                   ...
-0.08205777488270032  0.9184797375869633    0.03188119325449726   ...
0.28575755805713965   0.6307494802500775    -0.08882582642644302  ...


In this example, we don't have many variables descriptives, so we suppose that we can use all variables to build the regression model. Otherwise, we need to do a selection of variables to select the variables that affect the most the target variable. To do selection variable, depending on the type of variables, we can use different methods. In Spark, we have some basic tools to do that, for example https://spark.apache.org/docs/latest/ml-features.html#feature-selectors 

###  Vector Assembler

To prepare for the construction of linear regression by using ML library, we have to have a data with 2 columns only ("label" and "features"). To have that, we need to put all the variables descriptives into a single vector column named "features" and column of the target variable should be renamed to "label". 

In [ ]:
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors

In [ ]:
val assembler = new VectorAssembler()
  .setInputCols(Array("Temperature", "Production", "Days", "Persons"))
  .setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_c2bfdad4cf23


In [ ]:
val training = assembler.transform(data).select("Water", "features")
.withColumnRenamed("Water", "label")

### Build a linear regression model 

To have the best model, we can try to fluctuate the parameters such as : number of max iterations, regularization parameters, etc. To find all the parameters supported by Spark that we can play with, you can see it in : https://spark.apache.org/docs/1.6.2/api/scala/index.html#org.apache.spark.ml.regression.LinearRegression

In [ ]:
import org.apache.spark.ml.regression.LinearRegression

val lr = new LinearRegression()
  .setMaxIter(10)
  .setRegParam(0.3)
  .setElasticNetParam(0.8)

// Fit the model
val lrModel = lr.fit(training)

In [ ]:
// Print the coefficients and intercept for linear regression
println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

### Evaluation of model 

Some other metrics that can be computed : https://spark.apache.org/docs/1.6.2/api/scala/index.html#org.apache.spark.ml.regression.LinearRegressionTrainingSummary

In [ ]:
// Summarize the model over the training set and print out some metrics
val trainingSummary = lrModel.summary
println(s"numIterations: ${trainingSummary.totalIterations}")
println(s"objectiveHistory: [${trainingSummary.objectiveHistory.mkString(",")}]")
trainingSummary.residuals.show()
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")
println(s"r2: ${trainingSummary.r2}")

### Conclusion

Without any optimization, the quality of the model is pretty good (r2 = 0.76). In reality, we can try to optimize this indicator by removing the anomalies, selecting the most important features to train model, adding more observations or more variables and fluctuating the parameters when we train model...

### Note :

All models created in Spark can be saved in HDFS by doing : 

* model.save(sc, "file:///Apps/spark/data/mllib/testModelPath") 

To load it for future usage : 

* val sameModel = SVMModel.load(sc, "file:///Apps/spark/data/mllib/testModelPath"). 

In this example, it's SVM model, so it's SVMModel.load

Plus, for some models, we can convert it to PMML format. It's good if you knew already PMML, if not, it's also fine ;) you can read here : https://www.ibm.com/developerworks/library/ba-ind-PMML1/index.html.

You can see list of supported models in Spark here : https://spark.apache.org/docs/2.0.0-preview/mllib-pmml-model-export.html